In [23]:
import os
import json
import torch
from collections import OrderedDict
from transformers import AutoConfig, AutoTokenizer, PreTrainedTokenizerFast
from types import SimpleNamespace

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def check_for_weight_keys(ckpt):
    new_state_dict = OrderedDict()
    for k, v in ckpt.items():
        if k[:6] == 'module':               
            name = k[7:] # remove `module.`
        else:
            name = k
        new_state_dict[name] = v
    return new_state_dict

In [34]:
from lm_eval.models.opt_models.dynamic_opt import OPTForCausalLM

tokenizer = PreTrainedTokenizerFast(tokenizer_file='./lm_eval/20B_tokenizer.json')
tokenizer.pad_token_id = 1

with open('/home/youpengzhao/code/pretrained/60M_150k/config.json') as f:
    data = json.loads(f.read())
config = SimpleNamespace(**data)        
# change vocab size to match 20B tokenizer
config.vocab_size = 50277
opt = OPTForCausalLM(config)
ckpt = torch.load('/home/youpengzhao/code/pretrained/60M_150k/opt_60000.pth', map_location='cpu')
# load_pretrained(ckpt, self.opt)
ckpt = check_for_weight_keys(ckpt)
opt.load_state_dict(ckpt)
print('Load Pretrained Weights')
print_size_of_model(opt)

Load Pretrained Weights
Size (MB): 237.203603


In [35]:
opt = torch.quantization.quantize_dynamic(
                opt, 
                {torch.nn.Linear}, 
                dtype=torch.qint8)

print_size_of_model(opt)

Size (MB): 217.121468


In [ ]:
rho = 2/9

x = (1 + 3 * rho + 9 * rho^2 / 4 + 9 * rho^3/(2*(1-rho)))